In [ ]:
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.discriminant_analysis import StandardScaler
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
keras = tf.keras
from keras import Sequential
from keras.layers import Dense, InputLayer, Dropout
from keras.optimizers import SGD
from scikeras.wrappers import KerasClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# fetch dataset
heart_disease = fetch_ucirepo(id=45)

# extract data and target
X = heart_disease.data.features  # Features
y = heart_disease.data.targets   # Target variable

# targets that are 1-4, are now just equal to 1.
y = (y > 0).astype(int)

# select the 13 features (update these indices or column names based on the dataset)
selected_features = [
    "age", "sex", "cp", "trestbps", "chol", "fbs", "restecg",
    "thalach", "exang", "oldpeak", "slope", "ca", "thal"
]

# using only the selected features
X = X[selected_features]

# handle missing values by imputing them with the column mean
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
def build_model(num_hidden_layers=3, num_neurons=32, dropout_rate=0.0, momentum=0.0):
    model = Sequential()
    model.add(InputLayer(shape=(X_scaled.shape[1],)))
    model.add(Dropout(rate=dropout_rate))
    for _ in range(num_hidden_layers):
        model.add(Dense(num_neurons, activation='relu')) # Hidden layer using ReLU activation function
        model.add(Dropout(rate=dropout_rate))
    model.add(Dense(1, activation='sigmoid')) # Output layer using sigmoid activation function
    optimizer = SGD(learning_rate=0.01, momentum=momentum)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

param_grid = {
    'model__num_hidden_layers': [1, 2, 3],
    'model__num_neurons': [64, 128],
    'model__dropout_rate': [0.1, 0.3],
    'model__momentum': [0.5, 0.9]
}

model = KerasClassifier(model=build_model, epochs=50, batch_size=10, verbose=0)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=kfold, verbose=5) # Grid search with 5-fold cross-validation
grid_search.fit(X_scaled, y)

# Visualize the results of the grid search
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Mean: {mean}, Stdev: {stdev} with: {param}")


In [ ]:
# Get the best model
print(f"Best: {grid_search.best_score_} using {grid_search.best_params_}")
best_model = grid_search.best_estimator_.model
best_model.save("ann_best_model.keras")

In [ ]:
# Split the data into a test set to evaluate the best model
from sklearn.model_selection import train_test_split

_, X_test, _, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)

# Get y_pred from the best model
y_pred = best_model.predict(X_test)

# Convert the probabilities to binary predictions
binary_y_pred = (y_pred > 0.5).astype(int)

# Compute the confusion matrix
cm = confusion_matrix(y_test, binary_y_pred)

# Extract TN, FP, FN, and TP
tn, fp, fn, tp = cm.ravel()

# Calculate metrics
precision = precision_score(y_test, binary_y_pred)
recall = recall_score(y_test, binary_y_pred)
f1 = f1_score(y_test, binary_y_pred)
specificity = tn / (tn + fp)
roc_auc = roc_auc_score(y_test, y_pred)

# Print all metrics
print(f"Precision: {precision:.2f}")
print(f"Recall (Sensitivity): {recall:.2f}")
print(f"Specificity: {specificity:.2f}")
print(f"F1-Score: {f1:.2f}")

accuracy = (tp + tn) / (tp + tn + fp + fn)
print(f"Accuracy: {accuracy:.2f}")

# Generate and display Confusion Matrix
ConfusionMatrixDisplay.from_predictions(y_test, binary_y_pred, display_labels=["No Disease", "Disease"], cmap="Blues")
plt.title("Confusion Matrix")
plt.show()

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
plt.figure()
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], "k--", label="Random Guess")
plt.xlabel("False Positive Rate (1 - Specificity)")
plt.ylabel("True Positive Rate (Recall)")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="best")
plt.show()